In [12]:
#Importing Required Libraries
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
import dask.dataframe as dk
import calendar
from sklearn.metrics import mean_squared_error
from tqdm import tqdm
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler,Normalizer
from scipy.sparse import csr_matrix,hstack
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import SGDRegressor
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
from sklearn.ensemble import AdaBoostRegressor
import pickle


In [14]:
#this snippet of code is used to reduce memory consumption by dataframe
dtype={'id'       :     'object', 
    'item_id'     :  'int64', 
    'dept_id'     :  'int8', 
    'cat_id'      :  'int8', 
    'store_id'    :  'int8', 
    'state_id'    :  'int8', 
    'd'           :  'object', 
    'sales'       :  'int16',  
    'date'        : 'object', 
   'wday'        :  'int8',  
   'month'       :  'int8',  
   'year'        :  'int16',  
   'event_name_1' : 'int8', 
   'event_type_1' : 'int8', 
   'event_name_2' : 'int8', 
   'event_type_2' : 'int8', 
    'snap':'int8',
  'sell_price'   : 'float16',
       'price_change':'float16',
   'week_number'  : 'int8',  
   'season'       : 'object', 
   'quater_start' : 'int8',  
   'quater_end'   : 'int8',  
   'month_start'  : 'int8',  
   'month_end'    : 'int8',  
   'year_start'   : 'int8',  
   'year_end'     : 'int8',  
   'group'        : 'int8',  
   'no_events'    : 'object', 
   'holiday'      : 'object',
    'week_number':'int8',
       'season':'int8',
       'quater_start':'int8',
       'quater_end':'int8',
       'month_start':'int8',
       'month_end':'int8',
       'year_start':'int8',
       'year_end':'int8',
       'roll_7_shift_28_mean':'float16',
       'roll_14_shift_28_mean':'float16',
       'roll_30_shift_28_mean':'float16',
       'roll_60_shift_28_mean':'float16',
       'roll_360_shift_28_mean':'float16',
       'roll_7_shift_28_std':'float16',
       'roll_14_shift_28_std':'float16',
       'roll_30_shift_28_std':'float16',
       'roll_60_shift_28_std':'float16',
       'roll_360_shift_28_std':'float16',
       'direct_ewm':'float16',
       'direct_lag_28':'int16',
       'direct_lag_35':'int16',
       'direct_lag_42':'int16',
       'direct_lag_49':'int16',
       'direct_lag_56':'int16',
       'direct_lag_63':'int16',
       'direct_lag_70':'int16',
       'direct_lag_77':'int16',
       'direct_lag_84':'int16',
       'direct_lag_91':'int16',
       'direct_lag_98':'int16',
       'min_price':'float16',
       'max_price':'float16',
       'mean_price':'float16',
       'std_price':'float16',
       'price_norm_1':'float16',
       'price_norm_2':'float16',
       'price_norm_3':'float16',
      }


In [17]:
train=pd.read_feather('/Users/shreyabharti/Desktop/Predictive Anaytics/Project/feather/train1.feather')
cv=pd.read_feather('/Users/shreyabharti/Desktop/Predictive Anaytics/Project/feather/cv1.feather')
test=pd.read_feather('/Users/shreyabharti/Desktop/Predictive Anaytics/Project/feather/test1.feather')
final_test=pd.read_feather('/Users/shreyabharti/Desktop/Predictive Anaytics/Project/feather/final_test1.feather')

In [20]:
def caluclate_WRMSSE(actual,predicted,train,weights,h,n):
    '''This function is used to calculate RMSSE'''
    num=((actual-predicted)**2).sum(axis=1)/h
    denom=(train[:,1:]-train[:,:-1])**2
    denom=denom.sum(axis=1)/(n-1)
    return (num/denom)**0.5

In [42]:
def get_model_performances(model, train, cv, test, X_cv, X_test):
    import gc

    # ========== CV WRMSSE ==========
    cv['prices'] = cv['sales'] * cv['sell_price']
    total_sales = cv.groupby('id').sum()
    total = total_sales['sales'].sum()
    weight = {i: total_sales.loc[i]['sales'] / total for i in total_sales.index}

    train1 = pd.concat([train, cv])
    train1.sort_values(['id', 'date'], inplace=True)
    train1.fillna(0, inplace=True)

    df = train1.pivot_table(index=['id', 'state_id', 'store_id', 'cat_id', 'dept_id', 'item_id'],
                            columns='d', values='sales').reset_index()
    df.fillna(0, inplace=True)
    del train1

    cv['pred_sales'] = model.predict(X_cv)
    df1 = cv.pivot_table(index='id', columns='d', values='pred_sales')
    df1.rename(columns={f'd_{i}': f'F{j+1}' for j, i in enumerate(range(1886, 1914))}, inplace=True)
    df1.reset_index(inplace=True)

    dd = df.merge(df1, on='id')
    dd['weight'] = dd['id'].apply(lambda x: weight[x])
    l = [f'd_{i}' for i in range(1, 1914)]
    l1 = [f'F{i}' for i in range(1, 29)]

    agg_level = {
        2: ['state_id'], 3: ['store_id'], 4: ['cat_id'], 5: ['dept_id'],
        6: ['state_id', 'cat_id'], 7: ['state_id', 'dept_id'],
        8: ['store_id', 'cat_id'], 9: ['store_id', 'dept_id'],
        10: ['item_id'], 11: ['item_id', 'state_id']
    }

    agg = pd.DataFrame(dd[l + l1].sum()).T
    agg['weight'] = 1 / 12
    agg['level'] = 1
    col = agg.columns

    for level in agg_level:
        temp_df = dd.groupby(by=agg_level[level]).sum().reset_index(drop=True)
        temp_df['weight'] /= 12
        temp_df['level'] = level
        agg = pd.concat([agg, temp_df[col]], ignore_index=True)

    dd['weight'] /= 12
    dd['level'] = 12
    agg = pd.concat([agg, dd[col]], ignore_index=True)

    actual = agg[[f'd_{i}' for i in range(1886, 1914)]].values
    predicted = agg[[f'F{i}' for i in range(1, 29)]].values
    training = agg[[f'd_{i}' for i in range(1, 1886)]].values
    weights = agg['weight'].values

    rmsse = caluclate_WRMSSE(actual, predicted, training, weights, 28, 1886)
    cv_wrsmme = np.sum(rmsse * weights)

    cv.drop(['pred_sales'], axis=1, inplace=True)
    del actual, predicted, training, weights, agg

    # ========== Test WRMSSE ==========
    test['prices'] = test['sales'] * test['sell_price']
    total_sales = test.groupby('id').sum()
    total = total_sales['sales'].sum()
    weight = {i.replace('evaluation', 'validation'): total_sales.loc[i]['sales'] / total for i in total_sales.index}

    test['id'] = test['id'].str.replace('evaluation', 'validation')
    train1 = pd.concat([train, cv, test])
    train1.sort_values(['id', 'date'], inplace=True)
    train1.fillna(0, inplace=True)

    df = train1.pivot_table(index=['id', 'state_id', 'store_id', 'cat_id', 'dept_id', 'item_id'],
                            columns='d', values='sales').reset_index()
    df.fillna(0, inplace=True)
    del train1

    test['pred_sales'] = model.predict(X_test)
    df1 = test.pivot_table(index='id', columns='d', values='pred_sales')
    df1.rename(columns={f'd_{i}': f'F{j+1}' for j, i in enumerate(range(1914, 1942))}, inplace=True)
    df1.reset_index(inplace=True)

    dd = df.merge(df1, on='id')
    dd['weight'] = dd['id'].apply(lambda x: weight[x])
    l = [f'd_{i}' for i in range(1, 1942)]
    l1 = [f'F{i}' for i in range(1, 29)]

    agg = pd.DataFrame(dd[l + l1].sum()).T
    agg['weight'] = 1 / 12
    agg['level'] = 1
    col = agg.columns

    for level in agg_level:
        temp_df = dd.groupby(by=agg_level[level]).sum().reset_index(drop=True)
        temp_df['weight'] /= 12
        temp_df['level'] = level
        agg = pd.concat([agg, temp_df[col]], ignore_index=True)

    dd['weight'] /= 12
    dd['level'] = 12
    agg = pd.concat([agg, dd[col]], ignore_index=True)

    actual = agg[[f'd_{i}' for i in range(1914, 1942)]].values
    predicted = agg[[f'F{i}' for i in range(1, 29)]].values
    training = agg[[f'd_{i}' for i in range(1, 1914)]].values
    weights = agg['weight'].values

    rmsse = caluclate_WRMSSE(actual, predicted, training, weights, 28, 1914)
    test_wrsmme = np.sum(rmsse * weights)

    print("CV WRMSSE =", cv_wrsmme)
    print("Test WRMSSE =", test_wrsmme)


In [28]:
!pip install tensorflow
import tensorflow as tf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.7/252.7 MB 10.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 28.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 26.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.8/25.8 MB 28.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.4/670.4 kB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 27.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 342.6/342.6 kB 31.5 MB/s eta 0:00:00


In [30]:
#Code Snippet to prepare data for serveral Models in which we pass sparse matrix
#converting item_id into string so that it can be used by Count Vectorizer
train['item_id']=train['item_id'].apply(lambda x:str(x))
cv['item_id']=cv['item_id'].apply(lambda x:str(x))
test['item_id']=test['item_id'].apply(lambda x:str(x))
final_test['item_id']=final_test['item_id'].apply(lambda x:str(x))


vec_item_id=CountVectorizer(binary=True)
train_id=vec_item_id.fit_transform(train['item_id'])
train_season=csr_matrix(tf.keras.utils.to_categorical(train['season'],4))
train_month=csr_matrix(tf.keras.utils.to_categorical(train['month'],13))
train_wday=csr_matrix(tf.keras.utils.to_categorical(train['wday'],8))
train_snap=csr_matrix(tf.keras.utils.to_categorical(train['snap'],2))
train_dept=csr_matrix(tf.keras.utils.to_categorical(train['dept_id'],7))
train_en1=csr_matrix(tf.keras.utils.to_categorical(train['event_name_1'],31))
train_en2=csr_matrix(tf.keras.utils.to_categorical(train['event_name_2'],5))
train_et1=csr_matrix(tf.keras.utils.to_categorical(train['event_type_1'],5))
train_et2=csr_matrix(tf.keras.utils.to_categorical(train['event_type_2'],3))
train_yr=csr_matrix(tf.keras.utils.to_categorical(train['year'],6))
train_yr_st=csr_matrix(tf.keras.utils.to_categorical(train['year_start'],2))
train_yr_end=csr_matrix(tf.keras.utils.to_categorical(train['year_end'],2))
train_qt_st=csr_matrix(tf.keras.utils.to_categorical(train['quater_start'],2))
train_qt_end=csr_matrix(tf.keras.utils.to_categorical(train['quater_end'],2))
train_month_st=csr_matrix(tf.keras.utils.to_categorical(train['month_start'],2))
train_month_end=csr_matrix(tf.keras.utils.to_categorical(train['month_end'],2))

cv_id=vec_item_id.transform(cv['item_id'])
cv_season=csr_matrix(tf.keras.utils.to_categorical(cv['season'],4))
cv_month=csr_matrix(tf.keras.utils.to_categorical(cv['month'],13))
cv_wday=csr_matrix(tf.keras.utils.to_categorical(cv['wday'],8))
cv_snap=csr_matrix(tf.keras.utils.to_categorical(cv['snap'],2))
cv_dept=csr_matrix(tf.keras.utils.to_categorical(cv['dept_id'],7))
cv_en1=csr_matrix(tf.keras.utils.to_categorical(cv['event_name_1'],31))
cv_en2=csr_matrix(tf.keras.utils.to_categorical(cv['event_name_2'],5))
cv_et1=csr_matrix(tf.keras.utils.to_categorical(cv['event_type_1'],5))
cv_et2=csr_matrix(tf.keras.utils.to_categorical(cv['event_type_2'],3))
cv_yr=csr_matrix(tf.keras.utils.to_categorical(cv['year'],6))
cv_yr_st=csr_matrix(tf.keras.utils.to_categorical(cv['year_start'],2))
cv_yr_end=csr_matrix(tf.keras.utils.to_categorical(cv['year_end'],2))
cv_qt_st=csr_matrix(tf.keras.utils.to_categorical(cv['quater_start'],2))
cv_qt_end=csr_matrix(tf.keras.utils.to_categorical(cv['quater_end'],2))
cv_month_st=csr_matrix(tf.keras.utils.to_categorical(cv['month_start'],2))
cv_month_end=csr_matrix(tf.keras.utils.to_categorical(cv['month_end'],2))


test_id=vec_item_id.transform(test['item_id'])
test_season=csr_matrix(tf.keras.utils.to_categorical(test['season'],4))
test_month=csr_matrix(tf.keras.utils.to_categorical(test['month'],13))
test_wday=csr_matrix(tf.keras.utils.to_categorical(test['wday'],8))
test_snap=csr_matrix(tf.keras.utils.to_categorical(test['snap'],2))
test_dept=csr_matrix(tf.keras.utils.to_categorical(test['dept_id'],7))
test_en1=csr_matrix(tf.keras.utils.to_categorical(test['event_name_1'],31))
test_en2=csr_matrix(tf.keras.utils.to_categorical(test['event_name_2'],5))
test_et1=csr_matrix(tf.keras.utils.to_categorical(test['event_type_1'],5))
test_et2=csr_matrix(tf.keras.utils.to_categorical(test['event_type_2'],3))
test_yr=csr_matrix(tf.keras.utils.to_categorical(test['year'],6))
test_yr_st=csr_matrix(tf.keras.utils.to_categorical(test['year_start'],2))
test_yr_end=csr_matrix(tf.keras.utils.to_categorical(test['year_end'],2))
test_qt_st=csr_matrix(tf.keras.utils.to_categorical(test['quater_start'],2))
test_qt_end=csr_matrix(tf.keras.utils.to_categorical(test['quater_end'],2))
test_month_st=csr_matrix(tf.keras.utils.to_categorical(test['month_start'],2))
test_month_end=csr_matrix(tf.keras.utils.to_categorical(test['month_end'],2))


final_test_id=vec_item_id.transform(final_test['item_id'])
final_test_season=csr_matrix(tf.keras.utils.to_categorical(final_test['season'],4))
final_test_month=csr_matrix(tf.keras.utils.to_categorical(final_test['month'],13))
final_test_wday=csr_matrix(tf.keras.utils.to_categorical(final_test['wday'],8))
final_test_snap=csr_matrix(tf.keras.utils.to_categorical(final_test['snap'],2))
final_test_dept=csr_matrix(tf.keras.utils.to_categorical(final_test['dept_id'],7))
final_test_en1=csr_matrix(tf.keras.utils.to_categorical(final_test['event_name_1'],31))
final_test_en2=csr_matrix(tf.keras.utils.to_categorical(final_test['event_name_2'],5))
final_test_et1=csr_matrix(tf.keras.utils.to_categorical(final_test['event_type_1'],5))
final_test_et2=csr_matrix(tf.keras.utils.to_categorical(final_test['event_type_2'],3))
final_test_yr=csr_matrix(tf.keras.utils.to_categorical(final_test['year'],6))
final_test_yr_st=csr_matrix(tf.keras.utils.to_categorical(final_test['year_start'],2))
final_test_yr_end=csr_matrix(tf.keras.utils.to_categorical(final_test['year_end'],2))
final_test_qt_st=csr_matrix(tf.keras.utils.to_categorical(final_test['quater_start'],2))
final_test_qt_end=csr_matrix(tf.keras.utils.to_categorical(final_test['quater_end'],2))
final_test_month_st=csr_matrix(tf.keras.utils.to_categorical(final_test['month_start'],2))
final_test_month_end=csr_matrix(tf.keras.utils.to_categorical(final_test['month_end'],2))

X_train=hstack((train_id,train_dept,train_month,train_wday,train_yr,\
                train_snap,train_en1,train_en2,train_et1,train_et2,train_yr_st,train_yr_end,
                train_qt_st,train_qt_end,train_month_st,train_month_end,\
                train['direct_lag_28'].values.reshape(-1,1),
                train['direct_lag_35'].values.reshape(-1,1),train['direct_lag_42'].values.reshape(-1,1),\
                train['direct_lag_49'].values.reshape(-1,1),train['direct_lag_56'].values.reshape(-1,1),\
                train['direct_lag_63'].values.reshape(-1,1),train['direct_lag_70'].values.reshape(-1,1),\
                train['direct_lag_77'].values.reshape(-1,1),train['direct_lag_84'].values.reshape(-1,1),\
                train['direct_lag_91'].values.reshape(-1,1),train['direct_lag_98'].values.reshape(-1,1),\
                train['sell_price'].values.reshape(-1,1))).tocsr()
y_train=train['sales'].values

X_cv=hstack((cv_id,cv_dept,cv_month,cv_wday,cv_yr,\
                cv_snap,cv_en1,cv_en2,cv_et1,cv_et2,cv_yr_st,cv_yr_end,
                cv_qt_st,cv_qt_end,cv_month_st,cv_month_end,\
             cv['direct_lag_28'].values.reshape(-1,1),
                cv['direct_lag_35'].values.reshape(-1,1),cv['direct_lag_42'].values.reshape(-1,1),\
                cv['direct_lag_49'].values.reshape(-1,1),cv['direct_lag_56'].values.reshape(-1,1),\
                cv['direct_lag_63'].values.reshape(-1,1),cv['direct_lag_70'].values.reshape(-1,1),\
                cv['direct_lag_77'].values.reshape(-1,1),cv['direct_lag_84'].values.reshape(-1,1),\
                cv['direct_lag_91'].values.reshape(-1,1),cv['direct_lag_98'].values.reshape(-1,1),\
                cv['sell_price'].values.reshape(-1,1)
                )).tocsr()
y_cv=cv['sales'].values


X_test=hstack((test_id,test_dept,test_month,test_wday,test_yr,\
                test_snap,test_en1,test_en2,test_et1,test_et2,test_yr_st,test_yr_end,
                test_qt_st,test_qt_end,test_month_st,test_month_end,\
                test['direct_lag_28'].values.reshape(-1,1),
                test['direct_lag_35'].values.reshape(-1,1),test['direct_lag_42'].values.reshape(-1,1),\
                test['direct_lag_49'].values.reshape(-1,1),test['direct_lag_56'].values.reshape(-1,1),\
                test['direct_lag_63'].values.reshape(-1,1),test['direct_lag_70'].values.reshape(-1,1),\
                test['direct_lag_77'].values.reshape(-1,1),test['direct_lag_84'].values.reshape(-1,1),\
                test['direct_lag_91'].values.reshape(-1,1),test['direct_lag_98'].values.reshape(-1,1),\
                test['sell_price'].values.reshape(-1,1))).tocsr()
y_test=test['sales'].values

X_final_test=hstack((final_test_id,final_test_dept,final_test_month,final_test_wday,final_test_yr,\
                final_test_snap,final_test_en1,final_test_en2,final_test_et1,final_test_et2,final_test_yr_st,final_test_yr_end,
                final_test_qt_st,final_test_qt_end,final_test_month_st,final_test_month_end,\
               final_test['direct_lag_28'].values.reshape(-1,1),
                final_test['direct_lag_35'].values.reshape(-1,1),final_test['direct_lag_42'].values.reshape(-1,1),\
                final_test['direct_lag_49'].values.reshape(-1,1),final_test['direct_lag_56'].values.reshape(-1,1),\
                final_test['direct_lag_63'].values.reshape(-1,1),final_test['direct_lag_70'].values.reshape(-1,1),\
                final_test['direct_lag_77'].values.reshape(-1,1),final_test['direct_lag_84'].values.reshape(-1,1),\
                final_test['direct_lag_91'].values.reshape(-1,1),final_test['direct_lag_98'].values.reshape(-1,1),\
                final_test['sell_price'].values.reshape(-1,1))).tocsr()
y_final_test=final_test['sales'].values


In [81]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np


reg = LinearRegression(n_jobs=7)  # ← removed normalize
reg.fit(X_train, y_train)
y_pred_tr = reg.predict(X_train)
y_pred_cv = reg.predict(X_cv)

error_tr = np.sqrt(mean_squared_error(y_train, y_pred_tr))
error_tst = np.sqrt(mean_squared_error(y_cv, y_pred_cv))

print("Linear Regression model")
print("=" * 15)
print("Train RMSE =", error_tr, " CV RMSE =", error_tst)


Linear Regression model
Train RMSE = 2.8463633756840157  CV RMSE = 2.214157611164102


In [82]:
get_model_performances(reg,train,cv,test,X_cv,X_test)

CV WRMSSE = 0.7767322972180203
Test WRMSSE = 1.1099952527131942


In [83]:
test['pred_sales']=reg.predict(X_test)
final_test['pred_sales']=reg.predict(X_final_test)

In [84]:
#reg = make_pipeline(StandardScaler(), LinearRegression(n_jobs=7))
#reg.fit(X_train, y_train)


In [85]:
df=test.pivot_table(index='id',columns='date',values='pred_sales')
df.reset_index(level=0,inplace=True)
df['id']=df['id'].apply(lambda x:x.replace('evaluation','validation'))
dic={}
l=test['date'].unique()
for i,day in enumerate(l):
    dic[day]='F'+str(i+1)
df.rename(columns=dic,inplace=True)  

In [86]:
df1=final_test.pivot_table(index='id',columns='d',values='pred_sales')
df1.reset_index(level=0,inplace=True)
dic={}
for i,day in enumerate(range(1942,1970)):
    dic['d_'+str(day)]='F'+str(i+1)
df1.rename(columns=dic,inplace=True)    

In [87]:
df2=pd.concat([df1,df])

In [88]:
df2.to_csv('linear_regression.csv',index=False)

In [60]:
reg=AdaBoostRegressor(n_estimators=5,learning_rate=0.1,loss='square')
reg.fit(X_train,y_train)
print("Adaboost Regression model")
y_pred_tr=reg.predict(X_train)
y_pred_cv=reg.predict(X_cv)
error_tr=np.sqrt(mean_squared_error(y_train,y_pred_tr))
error_tst=np.sqrt(mean_squared_error(y_cv,y_pred_cv))
print("="*15)
print("Train RMSE=",error_tr," CV RMSE=",error_tst)

Adaboost Regression model
Train RMSE= 3.2091373069033975  CV RMSE= 2.551286303614295


In [67]:
#Get Test and CV RMSSE
get_model_performances(reg,train,cv,test,X_cv,X_test)

CV WRMSSE = 1.1304881971688396
Test WRMSSE = 1.1795962492552754


In [68]:
get_model_performances(reg,train,cv,test,X_cv,X_test)

CV WRMSSE = 1.1304881971688396
Test WRMSSE = 1.1795962492552754


In [69]:
test['pred_sales']=reg.predict(X_test)
final_test['pred_sales']=reg.predict(X_final_test)

In [70]:
df=test.pivot_table(index='id',columns='date',values='pred_sales')
df.reset_index(level=0,inplace=True)
df['id']=df['id'].apply(lambda x:x.replace('evaluation','validation'))
dic={}
l=test['date'].unique()
for i,day in enumerate(l):
    dic[day]='F'+str(i+1)
df.rename(columns=dic,inplace=True)  

In [71]:
df1=final_test.pivot_table(index='id',columns='d',values='pred_sales')
df1.reset_index(level=0,inplace=True)
dic={}
for i,day in enumerate(range(1942,1970)):
    dic['d_'+str(day)]='F'+str(i+1)
df1.rename(columns=dic,inplace=True)    


In [72]:
df2=pd.concat([df1,df])

In [73]:
df2.to_csv('adaboost.csv',index=False)